<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO


For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py


In [ ]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [ ]:
#used to help some of the timing functions
now = datetime.datetime.now

In [ ]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [ ]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [ ]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [ ]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [ ]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [ ]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# Let's take a look
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [ ]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 29404 samples, validate on 4861 samples
Epoch 1/5


2023-09-08 14:23:01.508905: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-09-08 14:23:01.514499: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394300000 Hz
2023-09-08 14:23:01.515278: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55de9cb1a290 executing computations on platform Host. Devices:
2023-09-08 14:23:01.515340: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-09-08 14:23:01.638218: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

29404/29404 [==============================] - 90s 3ms/step - loss: 0.2250 - acc: 0.9237 - val_loss: 0.0797 - val_acc: 0.9728
Epoch 2/5
29404/29404 [==============================] - 84s 3ms/step - loss: 0.0695 - acc: 0.9780 - val_loss: 0.0350 - val_acc: 0.9870
Epoch 3/5
29404/29404 [==============================] - 86s 3ms/step - loss: 0.0499 - acc: 0.9848 - val_loss: 0.0280 - val_acc: 0.9901
Epoch 4/5
29404/29404 [==============================] - 86s 3ms/step - loss: 0.0392 - acc: 0.9880 - val_loss: 0.0238 - val_acc: 0.9914
Epoch 5/5
29404/29404 [==============================] - 85s 3ms/step - loss: 0.0325 - acc: 0.9894 - val_loss: 0.0211 - val_acc: 0.9926
Training time: 0:07:11.254055
Test score: 0.021119811442068332
Test accuracy: 0.9925941163265909


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.


In [ ]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.


In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 17s 549us/step - loss: 0.1554 - accuracy: 0.9577 - val_loss: 0.0177 - val_accuracy: 0.9938
Epoch 2/5
30596/30596 [==============================] - 17s 543us/step - loss: 0.0400 - accuracy: 0.9884 - val_loss: 0.0109 - val_accuracy: 0.9955
Epoch 3/5
30596/30596 [==============================] - 16s 536us/step - loss: 0.0282 - accuracy: 0.9917 - val_loss: 0.0094 - val_accuracy: 0.9973
Epoch 4/5
30596/30596 [==============================] - 16s 520us/step - loss: 0.0244 - accuracy: 0.9923 - val_loss: 0.0069 - val_accuracy: 0.9975
Epoch 5/5
30596/30596 [==============================] - 17s 549us/step - loss: 0.0188 - accuracy: 0.9945 - val_loss: 0.0065 - val_accuracy: 0.9973
Training time: 0:01:25.544194
Test score: 0.0064982816989409215
Test accuracy: 0.997275710105896


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster.


## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.


In [ ]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
__________

In [ ]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 87s 3ms/step - loss: 0.1653 - acc: 0.9467 - val_loss: 0.0254 - val_acc: 0.9912
Epoch 2/5
30596/30596 [==============================] - 87s 3ms/step - loss: 0.0469 - acc: 0.9862 - val_loss: 0.0151 - val_acc: 0.9944
Epoch 3/5
30596/30596 [==============================] - 88s 3ms/step - loss: 0.0332 - acc: 0.9905 - val_loss: 0.0112 - val_acc: 0.9963
Epoch 4/5
30596/30596 [==============================] - 87s 3ms/step - loss: 0.0253 - acc: 0.9921 - val_loss: 0.0094 - val_acc: 0.9971
Epoch 5/5
30596/30596 [==============================] - 86s 3ms/step - loss: 0.0218 - acc: 0.9934 - val_loss: 0.0080 - val_acc: 0.9971
Training time: 0:07:15.190551
Test score: 0.007952959932693307
Test accuracy: 0.997081144191477


In [ ]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)             

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [ ]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 22s 757us/step - loss: 0.2482 - accuracy: 0.9302 - val_loss: 0.0548 - val_accuracy: 0.9817
Epoch 2/5
29404/29404 [==============================] - 20s 678us/step - loss: 0.0802 - accuracy: 0.9754 - val_loss: 0.0369 - val_accuracy: 0.9881
Epoch 3/5
29404/29404 [==============================] - 20s 687us/step - loss: 0.0613 - accuracy: 0.9806 - val_loss: 0.0305 - val_accuracy: 0.9899
Epoch 4/5
29404/29404 [==============================] - 20s 679us/step - loss: 0.0506 - accuracy: 0.9843 - val_loss: 0.0274 - val_accuracy: 0.9901
Epoch 5/5
29404/29404 [==============================] - 20s 688us/step - loss: 0.0457 - accuracy: 0.9856 - val_loss: 0.0251 - val_accuracy: 0.9924
Training time: 0:01:48.137802
Test score: 0.025142969561196542
Test accuracy: 0.9923883676528931


---
### Machine Learning Foundation (C) 2020 IBM Corporation
